In [1]:
import osmnx as ox
import networkx as nx
import random
import pandas as pd
import folium
from shapely.geometry import LineString
from folium import Map, PolyLine
import sklearn


In [2]:
import geopandas as gpd

# Load saved edges GeoDataFrame
edges = gpd.read_file("C://Users\khong\Documents\Python Projects\Drive\Drive\data/edges_roads_with_curvature.gpkg", layer='edges')
nodes = gpd.read_file("C://Users\khong\Documents\Python Projects\Drive\Drive\data/nodes_roads_with_curvature.gpkg", layer='nodes')


<>:4: SyntaxWarning: invalid escape sequence '\k'
<>:5: SyntaxWarning: invalid escape sequence '\k'
<>:4: SyntaxWarning: invalid escape sequence '\k'
<>:5: SyntaxWarning: invalid escape sequence '\k'
C:\Users\khong\AppData\Local\Temp\ipykernel_9912\3870012126.py:4: SyntaxWarning: invalid escape sequence '\k'
  edges = gpd.read_file("C://Users\khong\Documents\Python Projects\Drive\Drive\data/edges_roads_with_curvature.gpkg", layer='edges')
C:\Users\khong\AppData\Local\Temp\ipykernel_9912\3870012126.py:5: SyntaxWarning: invalid escape sequence '\k'
  nodes = gpd.read_file("C://Users\khong\Documents\Python Projects\Drive\Drive\data/nodes_roads_with_curvature.gpkg", layer='nodes')


In [3]:
edges.set_index(['u', 'v', 'key'], inplace=True)
nodes.set_index(['osmid'], inplace=True)

In [4]:
# Reconstruct the graph
G = ox.graph_from_gdfs(nodes, edges)

In [8]:
start_lat = 53.1701   # Parsley Hay Car Park lat
start_lon = -1.7816  # Parsley Hay Car Park lon

start_node = ox.nearest_nodes(G, start_lon, start_lat)


In [46]:
import osmnx.distance

def get_random_waypoints(G, start_lat, start_lon, num_waypoints=3, radius=1000):
    start_node = ox.nearest_nodes(G, start_lon, start_lat)
    nodes = list(G.nodes(data=True))

    # Filter nodes within radius
    candidate_nodes = [
        (n, data) for n, data in nodes
        if osmnx.distance.euclidean(start_lat, start_lon, data['y'], data['x']) < radius
    ]

    if len(candidate_nodes) < num_waypoints:
        raise ValueError("Not enough nearby nodes to sample waypoints.")

    sampled = random.sample(candidate_nodes, num_waypoints)
    return [(data['y'], data['x']) for _, data in sampled]


# Generate waypoints
waypoints = get_random_waypoints(G, start_lat, start_lon, num_waypoints=5)

In [47]:
waypoints

[(np.float64(53.1391515), np.float64(-1.7825363)),
 (np.float64(53.2131391), np.float64(-1.677154)),
 (np.float64(53.1383667), np.float64(-1.7877553)),
 (np.float64(53.084546), np.float64(-1.7092295)),
 (np.float64(53.3081947), np.float64(-1.7593248))]

In [48]:
# --- Create Folium map ---
m = folium.Map(location=[start_lat, start_lon], zoom_start=14)

# Plot start
folium.Marker([start_lat, start_lon], popup="Start", icon=folium.Icon(color='green')).add_to(m)

# Plot waypoints
for idx, (lat, lon) in enumerate(waypoints, start=1):
    folium.Marker([lat, lon], popup=f"Waypoint {idx}", icon=folium.Icon(color='blue')).add_to(m)

# Show map
m